In [ ]:
import requests
from requests_html import HTMLSession
import arrow
from pathlib import Path
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=12, threads_per_worker=1)
client = Client(cluster)  # start local workers as processes

session = HTMLSession()
CWD = Path.cwd()
PDFS_DIR = CWD / "pdfs"
PDFS_DIR.mkdir(exist_ok=True)

In [ ]:
pdf_list_r = session.get(
    "https://travel.state.gov/content/travel/en/legal/visa-law0/visa-statistics/immigrant-visa-statistics/monthly-immigrant-visa-issuances.html"
)
pdf_list_content = pdf_list_r.html.find(".contentbody", first=True)
pdf_links = [
    link for link in pdf_list_content.find("a") if link.attrs["href"].endswith(".pdf")
]
niv_pdf_list_r = session.get(
    "https://travel.state.gov/content/travel/en/legal/visa-law0/visa-statistics/nonimmigrant-visa-statistics/monthly-nonimmigrant-visa-issuances.html"
)
niv_pdf_list_content = niv_pdf_list_r.html.find(".contentbody", first=True)
niv_pdf_links = [
    link
    for link in niv_pdf_list_content.find("a")
    if link.attrs["href"].endswith(".pdf")
]

In [ ]:
urls_by_month = {}
for link in [*pdf_links, *niv_pdf_links]:
    link_text = link.text.replace(" - ", " - ").replace(" – ", " - ")

    month, _, category = link_text.partition(" - ")
    category = category.replace("Visa Cass", "Visa Class")  # lol

    if "NIV Issuances by Post and Visa Class" in category:
        urls_by_month["NIV-" + arrow.get(month, "MMMM YYYY").format("YYYY-MM")] = list(
            link.absolute_links
        )[0]
    elif "IV Issuances by Post and Visa Class" in category:
        urls_by_month["IV-" + arrow.get(month, "MMMM YYYY").format("YYYY-MM")] = list(
            link.absolute_links
        )[0]

In [ ]:
from itertools import product

for kind, month in product(
    ["NIV", "IV"], arrow.Arrow.range("month", arrow.get("2017-03-01"), arrow.get())
):
    outfile = PDFS_DIR / f"{kind}-{month.format('YYYY-MM')}.pdf"
    if outfile.exists():
        print(f"skippy {outfile}")
        continue
    print(f"doin {outfile}")
    try:
        url = urls_by_month[f"{kind}-{month.format('YYYY-MM')}"]
    except KeyError:
        print(f"but no {outfile}!")
        continue
    r = requests.get(url)
    outfile.write_bytes(r.content)

In [ ]:
import camelot
import pandas as pd
import PyPDF2

client.restart()


def process_path(path: Path, page_no: int):
    parsed = camelot.read_pdf(str(path), pages=str(page_no))
    if len(parsed) == 0:
        return
    table = parsed[0].df
    table.columns = ["Post", "Visa Class", "Issuances"]
    table["Issuances"] = pd.to_numeric(
        table["Issuances"].replace(r"\D", "", regex=True),
        errors="coerce",
        downcast="integer",
    )
    table = table.replace("", pd.NA).dropna().reset_index(drop=True)
    table["Post"] = pd.Series(table["Post"], dtype="string")
    table["Visa Class"] = pd.Series(table["Visa Class"], dtype="string")
    table["Month"] = pd.Timestamp(
        arrow.get(
            path.name.replace("NIV-", "").replace("IV-", "").replace(".pdf", "")
        ).format("YYYYMMDD")
    )
    table["Issuances"] = table["Issuances"].astype("uint16")
    return table


futures = []
for path in PDFS_DIR.glob("*.pdf"):
    page_count = PyPDF2.PdfFileReader(path.open("rb")).getNumPages()
    for page_no in range(1, page_count + 1):
        futures.append(client.submit(process_path, path, page_no))

results = [result for result in client.gather(futures) if result is not None]
all_months = pd.concat(results).sort_values("Month").reset_index(drop=True)

In [ ]:
# cluster.scale(0)
cluster

In [ ]:
all_months[all_months["Issuances"] > 1000].head()

In [ ]:
all_months.to_pickle("all_months.pkl")

In [ ]:
bud_marriage = (
    all_months.loc[all_months["Post"] == "Budapest"]
    .loc[all_months["Visa Class"].isin(["CR1", "IR1"])]
    .groupby(by=all_months["Month"])
    .sum()
)
bud_marriage

In [ ]:
import seaborn as sns

sns.set_theme()

In [ ]:
bud_marriage["3mo_avg"] = bud_marriage["Issuances"].rolling(7).mean()
sns.lineplot(data=bud_marriage, x="Month", y="Issuances")
sns.lineplot(data=bud_marriage, x="Month", y="3mo_avg")

In [ ]:
(CWD / "bud.csv").write_text(bud_marriage.to_csv())